In [ ]:
import pandas as pd
from typing import Dict
from datetime import datetime
import pandas as pd
import datetime
from datetime import date, timedelta
import numpy as np
import matplotlib.pyplot as plt
from google.oauth2 import service_account
from googleapiclient.discovery import build, Resource
from IPython.display import display, Markdown

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Variablen anpassen

Welche Domain soll analysiert werden?

In [ ]:
############################################################################################
#Variables 
### Zunächst wird danach gefiltert, welche Suchbegriffe nicht enthalten sein sollen (notContains)
brand = "guru"

#AT(aut), Switzerland(che), Netherlands(nl), Spain(esp), Germany(deu)
COUNTRY_FILTER = ["esp"]

#DE(urlaubsguru.de), NLD(holidayguru.nl), ES(holidayguru.es), AT(urlaubsguru.at). CH(holidayguru.ch)
domain_name = "holidayguru.es" 

#Welche URLs möchtest du dir ansehen & wann wurden diese optimiert?
url_list = {
    "https://www.holidayguru.es/calendario-de-viajes/nochevieja/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-en-madrid/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-en-valencia/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-palma/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-malaga/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-en-sevilla/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-murcia/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/fin-de-ano-en-barcelona/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-bilbao/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-en-zaragoza/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/fin-ano-andorra/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-en-londres/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-roma/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/nochevieja-en-paris/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/fin-de-ano-en-praga-consejos-y-trucos/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/tradiciones-de-navidad-alrededor-del-mundo/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/mercadillos-navidenos-en-madrid/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/las-10-mejores-fiestas-populares-de-espaa/": "2023-11-29",
    "https://www.holidayguru.es/revista-de-viajes/paises-diferentes-culturas-diversas/": "2023-11-29"
}
############################################################################################

## Data Pipeline GSC

In diesem Bereich werden die Daten direkt aus der GSC gezogen. Dabei gibt es folgende zwei Zeiträume:
- Datum der Optimierung (siehe oben) bis heute
- Der exakt selbe Zeitraum vor der Optimierung

In [ ]:
#Variablen für den API Request
DIMENSIONS_BYURL = ["page", "query", "date", "country"]
DOMAIN = "sc-domain:" + domain_name
credential_filepath = ".secrets/creds.json"
MAX_ROWS = 25_000

#calculate date
today = date.today()
end_date = today

#Functions
#Für Timeframe Comparison:

def determine_timerange(row, comparison_start_date):
    if row["date"] > start_date:
        return "after"
    else:
        return "prior"

## für Auth
def auth_using_key_file(key_filepath):
    credentials = service_account.Credentials.from_service_account_file(
        key_filepath, scopes=SCOPE
    )
    service = build(API_SERVICE_NAME, API_VERSION, credentials=credentials)
    return service

def query(client: Resource, payload: Dict[str, str]) -> Dict[str, any]:
    response = client.searchanalytics().query(siteUrl=DOMAIN, body=payload).execute()
    return response

# Plot Function (wird mit Input aus der Loop gefüttert)
def plot_data(title, x_label, y_label, data_frame, url):
    fig, ax = plt.subplots(figsize=(10, 6))

    data_frame = data_frame[["prior", "after"]]
    for column in data_frame.columns:
        filtered_data = data_frame[data_frame[column] != 0]
        ax.plot(filtered_data.index, filtered_data[column], marker='o', label=column)

    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.legend()
    ax.grid(True)

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Plot Function (wird mit Input aus der Loop gefüttert)
def plot_data_inverted(title, x_label, y_label, data_frame, url):
    fig, ax = plt.subplots(figsize=(10, 6))

    data_frame = data_frame[["prior", "after"]]
    for column in data_frame.columns:
        filtered_data = data_frame[data_frame[column] != 0]
        ax.plot(filtered_data.index, filtered_data[column], marker='o', label=column)


    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.legend()
    ax.grid(True)

    plt.xticks(rotation=45)
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()

#metric difference calculation
def calculate_difference(df, url, query):
    prior_clicks = df[(df["page"] == url) & (df["timerange"] == "prior") & (df["query"] == query)]["clicks"].sum()
    after_clicks = df[(df["page"] == url) & (df["timerange"] == "after") & (df["query"] == query)]["clicks"].sum()

    prior_avg_position = df[(df["page"] == url) & (df["timerange"] == "prior") & (df["query"] == query)]["position"].mean()
    after_avg_position = df[(df["page"] == url) & (df["timerange"] == "after") & (df["query"] == query)]["position"].mean()

    clicks_difference = after_clicks - prior_clicks
    avg_position_difference = after_avg_position - prior_avg_position

    return clicks_difference, avg_position_difference, prior_clicks, after_clicks, prior_avg_position, after_avg_position

############################################################################################
# GSC API
API_SERVICE_NAME = "webmasters"
API_VERSION = "v3"
SCOPE = ["https://www.googleapis.com/auth/webmasters.readonly"]

KEY_FILE = credential_filepath
service = auth_using_key_file(key_filepath=KEY_FILE)
############################################################################################
#Beginn des API Requests
# dataframe um daten über die API zu speichern
data_frames = []

# by URL
for url, date in url_list.items():
    #start date calculatin
    start_date = datetime.datetime.strptime(date, "%Y-%m-%d").date()
    #comparison date range
    delta_days = (start_date - end_date).days
    delta_days *= -1
    comparison_start_date = start_date - timedelta(days=delta_days)
    comparison_end_date = end_date - timedelta(days=delta_days)
    start_date = start_date + timedelta(days=1)
    
    i = 0
    reponse_by_url = []
    while True:
        payload_main_range = {
            "startDate": start_date.strftime("%Y-%m-%d"),
            "endDate": end_date.strftime("%Y-%m-%d"),
            "dimensions": DIMENSIONS_BYURL,
            "dimensionFilterGroups": [{
                "filters": [{
                    "dimension": "page",
                    "expression": url
                },
                    {
                    "dimension": "country",
                    "operator": "contains",
                    "expression": COUNTRY_FILTER
                    }]
            }],
            "rowLimit": MAX_ROWS,
            "startRow": i * MAX_ROWS
        }

        # make request to API
        response_main_range = query(service, payload_main_range)

        # if there are rows in the response, append to the temporary list
        if response_main_range.get("rows"):
            reponse_by_url.extend(response_main_range["rows"])
            i += 1
        else:
            break

        print(f"Collected {len(reponse_by_url):,} rows (main range) for {url}.")
    
    # Create a DataFrame from the temporary list
    by_url_data = pd.DataFrame(reponse_by_url)
    by_url_data[DIMENSIONS_BYURL] = pd.DataFrame(by_url_data["keys"].tolist(), index=by_url_data.index)
    by_url_data = by_url_data.drop(columns="keys")

    # Add a new column "timerange" and populate it based on payload range
    by_url_data["date"] = pd.to_datetime(by_url_data["date"]).dt.date
    by_url_data["timerange"] = by_url_data.apply(lambda row: determine_timerange(row, comparison_start_date), axis=1)
    data_frames.append(by_url_data)

    a = 0
    response_by_comparison_url = []
    while True:
        payload_comparison_range = {
            "startDate": comparison_start_date.strftime("%Y-%m-%d"),
            "endDate": comparison_end_date.strftime("%Y-%m-%d"),
            "dimensions": DIMENSIONS_BYURL,
             "dimensionFilterGroups": [{
                "filters": [{
                    "dimension": "page",
                    "expression": url
                },
                    {
                    "dimension": "country",
                    "operator": "contains",
                    "expression": COUNTRY_FILTER
                    }]
            }],
            "rowLimit": MAX_ROWS,
            "startRow": a * MAX_ROWS
        }

        # make request to API
        response_comparison_range = query(service, payload_comparison_range)

        # if there are rows in the response, append to the temporary list
        if response_comparison_range.get("rows"):
            response_by_comparison_url.extend(response_comparison_range["rows"])
            a += 1
        else:
            break

        print(f"Collected {len(reponse_by_url):,} rows (comparison range) for {url}.")

    # Create a DataFrame from the temporary list
    by_url_data_response = pd.DataFrame(response_by_comparison_url)
    by_url_data_response[DIMENSIONS_BYURL] = pd.DataFrame(by_url_data_response["keys"].tolist(), index=by_url_data_response.index)
    by_url_data_response = by_url_data_response.drop(columns="keys")

    # Add a new column "timerange" and populate it based on payload range
    by_url_data_response["date"] = pd.to_datetime(by_url_data_response["date"]).dt.date
    by_url_data_response["timerange"] = by_url_data_response.apply(lambda row: determine_timerange(row, comparison_start_date), axis=1)

    # Append the data frame to the list
    data_frames.append(by_url_data_response)

# Concatenate all DataFrames in the list into a single DataFrame
all_data = pd.concat(data_frames, ignore_index=True)
df = all_data

## Data Manipulation

Aufbereitung der Daten mit dem Ziel:
- Zeiträume vor und nach der Optimierung zu aggregieren
- Ausspielen der Tabellen & Plots pro URL

Am Ende folgt die Erkenntnis:
- Das war das Top Keyword VOR der Optimierung
- Das war das Top Keyword NACH der Optimierung
- So haben sich Clicks, Query Count, Average Ranking des Top Keywords (nach Klicks) verändert

In [ ]:
#Manipulation
df = df[~df["query"].str.contains(brand)] # Brand Queries raushauen#
# Convert "date" column to datetime format
df["date"] = pd.to_datetime(df["date"])
# Create a new column "time_group" based on the year and week
#df['time_group'] = df['date'].dt.strftime('%Y-%U')
df['time_group'] = df['date'].dt.strftime('%Y-%m-%d')
# Assign a time label
time_label = "Year-Week"

for url, date in url_list.items():
    filtered_df = df[df["page"] == url]
    title_overall = f"## Performance Change für {url}\n\n"
    display(Markdown(title_overall))
    
    #click daten
    filtered_df_pivot_clicks = pd.pivot_table(
        filtered_df, 
        values="clicks", 
        index=["time_group"], 
        columns=["timerange"], 
        aggfunc=np.sum).fillna(0).astype(int)

    #query calculations
    diff_data = []
    top_queries_prior = filtered_df[filtered_df["timerange"] == "prior"].groupby("query")["clicks"].sum().nlargest(10)
    for query in top_queries_prior.index:
        clicks_diff, avg_position_diff, prior_clicks, after_clicks, prior_avg_position, after_avg_position = calculate_difference(filtered_df, url, query)
        diff_data.append({"Query": query, "Clicks Prior": prior_clicks, "Clicks After": after_clicks, "Clicks Difference": clicks_diff, "Position Prior": prior_avg_position, "Position After": after_avg_position, "Avg Position Difference": avg_position_diff})
    difference_df = pd.DataFrame(diff_data)
    difference_df["Avg Position Difference"] = difference_df["Avg Position Difference"].apply(lambda x: round(x, 2))
    difference_df["Position Prior"] = difference_df["Position Prior"].apply(lambda x: round(x, 2))
    difference_df["Position After"] = difference_df["Position After"].apply(lambda x: round(x, 2))
    title_markdown = "### Suchbegriffe, welche vor der Optimierung die meisten Klicks brachten, haben sich wie folgt verändert.\n\n"
    display(Markdown(title_markdown))
    display(difference_df)
    
    diff_data = []
    top_queries_after = filtered_df[filtered_df["timerange"] == "after"].groupby("query")["clicks"].sum().nlargest(10)
    for query in top_queries_after.index:
        clicks_diff, avg_position_diff, prior_clicks, after_clicks, prior_avg_position, after_avg_position = calculate_difference(filtered_df, url, query)
        diff_data.append({"Query": query, "Clicks Prior": prior_clicks, "Clicks After": after_clicks, "Clicks Difference": clicks_diff, "Position Prior": prior_avg_position, "Position After": after_avg_position, "Avg Position Difference": avg_position_diff})
    difference_df = pd.DataFrame(diff_data)
    difference_df["Avg Position Difference"] = difference_df["Avg Position Difference"].apply(lambda x: round(x, 2))
    difference_df["Position Prior"] = difference_df["Position Prior"].apply(lambda x: round(x, 2))
    difference_df["Position After"] = difference_df["Position After"].apply(lambda x: round(x, 2))
    title_markdown = "### Diese Suchbegriffe bringen nach der Optimierung die meisten Klicks.\n\n"
    display(Markdown(title_markdown))
    display(difference_df)  
    
    # Pivot table for query count over time
    filtered_df_forquerycount = filtered_df[filtered_df["clicks"] > 0]
    filtered_df_pivot_querycount = pd.pivot_table(
        filtered_df_forquerycount,
        values="query",
        index=["time_group"],
        columns=["timerange"],
        aggfunc="count"
    ).fillna(0).astype(int)

    #top keyword
    top_kw = filtered_df.groupby(by="query")["clicks"].sum().sort_values(ascending=False).head(1).index.tolist()
    top_kw_as_string = " ".join(top_kw)
    filtered_df_for_query = filtered_df[filtered_df["query"].isin(top_kw)]
    filtered_df_pivot_position = pd.pivot_table(
        filtered_df_for_query, 
        values="position", 
        index=["time_group"], 
        columns=["timerange"], 
        aggfunc=np.average).fillna(0).astype(float)
    
    #plotting
    plot_click_title = "### Visualisierung des Klick-Verlaufs"
    display(Markdown(plot_click_title))
    plot_data(
         title=f"{DOMAIN}: Clicks for \"{url}\"",
         x_label="Date",
         y_label="Sum of Clicks",
         data_frame=filtered_df_pivot_clicks,
         url=url
    )
    plot_qcount_title = "### Visualisierung des Query-Count-Verlaufs"
    display(Markdown(plot_qcount_title))

    plot_data(
        title=f"{DOMAIN}: Unique Query Count for URL \"{url}\"",
        x_label="Date",
        y_label="Unique Query Count (Clicks > 0)",
        data_frame=filtered_df_pivot_querycount,
        url=url
    )
    plot_avgposition_title = "### Visualisierung des Top-Keyword-Rankings"
    display(Markdown(plot_avgposition_title))
    plot_data_inverted(
        title=f"Average Ranking for \"{top_kw_as_string}\" | URL: \"{url}\"",
        x_label="Date",
        y_label="Average Ranking",
        data_frame=filtered_df_pivot_position,
        url=url
    )